# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [29]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [20]:
filelocation=os.path.join("..","output_data","cities.csv")
citiesdf=pd.read_csv(filelocation)
citiesdf=citiesdf[["City","Lat","Lng","Temperature (F)","Humidity (%)","Wind Speed (mph)","Cloudiness (%)"]]
citiesdf

,City,Lat,Lng,Temperature (F),Humidity (%),Wind Speed (mph),Cloudiness (%)
0,yulara,-25.24,130.99,86.00,23.0,14.99,50.0
1,praia da vitoria,38.73,-27.07,57.20,82.0,16.71,75.0
2,meulaboh,4.14,96.13,80.67,78.0,7.36,85.0
3,mawlaik,23.63,94.42,67.08,38.0,1.57,0.0
4,yarada,17.65,83.27,68.00,100.0,2.24,40.0
...,...,...,...,...,...,...,...
551,san pedro,14.35,121.02,83.70,61.0,11.41,20.0
552,takoradi,4.88,-1.76,77.76,90.0,3.04,99.0
553,longyan,25.11,117.02,79.00,99.0,3.40,100.0
554,bonthe,7.53,-12.51,79.14,83.0,3.89,100.0


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [82]:
gmaps.configure(api_key=g_key)
figure_layout = {
    'width': '720px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
citycoords=citiesdf[["Lat","Lng"]]
humidities=citiesdf["Humidity (%)"].astype(float)
humidities

0       23.0
1       82.0
2       78.0
3       38.0
4      100.0
       ...  
551     61.0
552     90.0
553     99.0
554     83.0
555     45.0
Name: Humidity (%), Length: 556, dtype: float64

In [83]:
heat_layer = gmaps.heatmap_layer(citycoords, weights=humidities)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [26]:
idealcitiesdf=citiesdf.loc[(citiesdf["Temperature (F)"]<80) & (citiesdf["Temperature (F)"]>60)&(citiesdf["Cloudiness (%)"]<20)&(citiesdf["Wind Speed (mph)"]<5)]
idealcitiesdf=idealcitiesdf.dropna()
idealcitiesdf

,City,Lat,Lng,Temperature (F),Humidity (%),Wind Speed (mph),Cloudiness (%)
3,mawlaik,23.63,94.42,67.08,38.0,1.57,0.0
51,champerico,14.30,-91.92,73.99,90.0,1.88,1.0
53,mandalay,21.97,96.08,72.66,39.0,1.36,0.0
98,saldanha,-33.01,17.94,66.20,63.0,2.24,0.0
101,makakilo city,21.35,-158.09,77.34,64.0,4.70,1.0
148,valdivia,-39.81,-73.25,62.60,82.0,2.24,0.0
157,harindanga,22.04,88.32,71.17,49.0,4.76,0.0
216,loei,17.42,101.50,68.18,93.0,2.24,2.0
236,ibra,22.69,58.53,60.85,84.0,4.36,17.0
281,chahal,15.75,-89.57,60.21,91.0,1.28,0.0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [72]:
url="https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
idealcitiesdf["Closest Hotel Name"]=None
hotels=[]
params={
    "key":g_key,
    "radius":"5000",
    "type":"lodging"
}
for index,row in idealcitiesdf.iterrows():    
    params['location']=str(row['Lat'])+","+str(row['Lng'])    
    response=requests.get(url,params=params).json()
#     print(response.url)
#     response=response.json()
#     pprint(response.json())
    try:
        idealcitiesdf.loc[index,"Closest Hotel Name"]=response["results"][0]["name"]
        location=(response["results"][0]["geometry"]["location"]["lat"],response["results"][0]["geometry"]["location"]["lng"])
        hotels.append({"Hotel Name":response["results"][0]["name"],"City":row["City"],"location":location,"Country":"test"})
    except(IndexError):
        print(row["City"])
    
    

chahal


In [73]:
idealcitiesdroppeddf=idealcitiesdf.dropna()
hotel

[{'name': 'AKZ Guest House',
  'City': 'mawlaik',
  'location': (23.6298756, 94.4168706)},
 {'name': 'Hotel y restaurante "El Diamante"',
  'City': 'champerico',
  'location': (14.2924083, -91.91331009999999)},
 {'name': 'Hotel Yadanarbon',
  'City': 'mandalay',
  'location': (21.9748786, 96.08998129999999)},
 {'name': 'Blue Bay Lodge',
  'City': 'saldanha',
  'location': (-32.995413, 17.97314)},
 {'name': "Marriott's Ko Olina Beach Club",
  'City': 'makakilo city',
  'location': (21.3330447, -158.1202393)},
 {'name': 'Hostal Entre Rios',
  'City': 'valdivia',
  'location': (-39.8100566, -73.2437468)},
 {'name': 'Sk giasuddin',
  'City': 'harindanga',
  'location': (22.049346, 88.3412877)},
 {'name': 'ไร่สานตม บ้านภูล้อม',
  'City': 'loei',
  'location': (17.4436139, 101.4739012)},
 {'name': 'مزرعة راحة النفوس المنزفة',
  'City': 'ibra',
  'location': (22.6862561, 58.5331392)},
 {'name': 'Platinum Hotel',
  'City': 'xai-xai',
  'location': (-25.0498155, 33.6387891)},
 {'name': 'Fisherm

In [75]:
hotel_locations = [hotel['location'] for hotel in hotels]
hotel_locations

[(23.6298756, 94.4168706),
 (14.2924083, -91.91331009999999),
 (21.9748786, 96.08998129999999),
 (-32.995413, 17.97314),
 (21.3330447, -158.1202393),
 (-39.8100566, -73.2437468),
 (22.049346, 88.3412877),
 (17.4436139, 101.4739012),
 (22.6862561, 58.5331392),
 (-25.0498155, 33.6387891),
 (-22.1335949, 14.2910675),
 (-28.55097280000001, 16.4236151),
 (-26.652957, 15.14962),
 (19.9053221, 109.6891395)]

In [76]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**hotel) for hotel in hotels]
locations = hotel_locations

In [84]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(hotel_locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display Map
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…